# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read previously stored data from csv to avoid repeaded api requests
citydata_df = pd.read_csv("output_data/cities.csv")
citydata_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.0752,-159.3190,26.18,76,90,0.89,US,1629912860
1,Esperance,-33.8667,121.9000,12.91,49,86,8.94,AU,1629912860
2,Qaanaaq,77.4840,-69.3632,4.72,85,100,4.78,GL,1629912861
3,Cape Town,-33.9258,18.4232,14.51,82,0,0.89,ZA,1629912861
4,Mehrān,33.1222,46.1646,36.71,9,0,4.29,IR,1629912861


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = citydata_df[["Lat", "Lng"]]
humidity = citydata_df["Humidity"].astype(float)

In [8]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
Tmax=(80-32)*5/9 #max temperature in C
Tmin=(70-32)*5/9
Wmax=10*0.44704  # max wind speed in m/s

citydata_ideal_df=citydata_df[(citydata_df['Max Temp']<Tmax) & (citydata_df['Max Temp']>Tmin) & (citydata_df['Wind Speed']<Wmax) & (citydata_df['Cloudiness']==0)]
citydata_ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Arrondissement d'Ajaccio,42.0000,8.9167,25.27,73,0,2.57,FR,1629912862
134,Ukrayinka,47.9167,35.3500,23.16,49,0,1.00,UA,1629912901
165,Luangwa,-15.6167,30.4167,25.94,27,0,2.89,ZM,1629912855
356,Gwanda,-20.9333,29.0000,22.26,29,0,4.15,ZW,1629912977
405,Mahajanga,-15.7167,46.3167,25.01,73,0,3.09,MG,1629912993
497,Dzhubga,44.3211,38.7039,24.45,78,0,1.87,RU,1629913023
517,Arrondissement de Saint-Denis,48.9167,2.3333,24.52,47,0,1.79,FR,1629913029
562,Khāsh,28.2211,61.2158,25.35,12,0,3.16,IR,1629913043


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
#
places_url_base="https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&locationbias=circle:5000@"


In [38]:
#Pull hotel names and their locations that are near the Ideal cities  
hotels_dict={'Hotel':[],'h_lat':[],'h_lng':[],'City':[],'Country':[]}
for index, row in citydata_ideal_df.iterrows():
    places_query_url=f"{places_url_base}{row['Lat']},{row['Lng']}&fields=name,geometry&key={g_key}"
    place_response = requests.get(places_query_url).json()
    hotels_dict['Hotel'].append(place_response['candidates'][0]['name'])
    hotels_dict['h_lat'].append(place_response['candidates'][0]['geometry']['location']['lat'])
    hotels_dict['h_lng'].append(place_response['candidates'][0]['geometry']['location']['lng'])
    hotels_dict['City'].append(row['City'])
    hotels_dict['Country'].append(row['Country'])

In [40]:
hotels_df=pd.DataFrame.from_dict(hotels_dict)
hotels_df.to_csv("output_data/hotels_cities_ideal.csv", index=False)
hotels_df

,Hotel,h_lat,h_lng,City,Country
0,Hôtel Mercure Ajaccio,41.932385,8.740392,Arrondissement d'Ajaccio,FR
1,Intourist hotel,47.839332,35.137341,Ukrayinka,UA
2,"Justfeeli Lodge, Luangwa",-15.625460,30.398416,Luangwa,ZM
3,Gwanda Hotel.,-20.943539,29.007666,Gwanda,ZW
4,Baobab Tree Hotel & Spa,-15.717707,46.301308,Mahajanga,MG
5,Marseille Hotel,44.301998,38.755036,Dzhubga,RU
6,Hôtel Régence Etoile,48.876537,2.292983,Arrondissement de Saint-Denis,FR
7,خوابگاه دانشجویی خاش,28.209650,61.198709,Khāsh,IR


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["h_lat", "h_lng"]]

In [43]:
# Add marker layer ontop of heat map
#fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))